In [6]:
import os
import numpy as np
from nibabel.testing import data_path
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import nibabel as nib
from tqdm import tqdm

In [2]:
def show(self, cmap=None):
        cmap = cmap or plt.cm.gray
        fig, ax = plt.subplots(1, 1, figsize=(8, 6))
        ax.imshow(self, cmap=cmap)
        ax.set_axis_off()
        #plt.show()
        return fig

### Załadowujemy trójwymiarową maskę z pliku .nii

In [3]:
#example_filename = os.path.join(data_path, 'example4d.nii.gz')
#img = nib.load('../../data/001_25_mm.nii')
#img_data = img.get_fdata()[:50,90:100,:50]
#del img
img_mask = nib.load('../../data/001_25_mask.nii')
img_mask_data = img_mask.get_fdata()#[:50,90:100,:50]
del img_mask


In [4]:
## zaloadowac maske i obraz
# obejrzec w 3D
# sprobowac 'wymazac' z maski wszystkie maski oprocz co piatej
# sprobowac interpolowac maske na 'wewnetrzne slice'y


In [ ]:
def interp_mine(img1, img2, dist):
    imgs = []
diff = (img_mask_data[:,0,:] - img_mask_data[:,5,:])
interp = (img_mask_data[:,0,:] + diff/5.0).astype(int) 
    return imgs

read image | denoised image
- | - 
{{x = show(img_data[:,9,:]) }}| {{y = show(img_mask_data[:,9,:])}}


In [5]:
# img_mask_data.shape

(232, 405, 280)

In [15]:
def prawidlowa_interpolacja_3D(img_mask_data, step=2):
    x_max, y_max, z_max = img_mask_data.shape
    grid_x, grid_y, grid_z = np.mgrid[0:x_max:1, 0:y_max:1,0:z_max:1]
    #points = np.random.rand(1000, 2)
    point_list = []
    for i in tqdm(range(x_max)):
        for j in range(0,y_max,2):
            for k in range(img_mask_data.shape[2]):
                point_list.append([i,j,k])
    points = np.array(point_list)
    values = img_mask_data[points[:,0],points[:,1],points[:,2]]
    grid_z2 = griddata(points, values, (grid_x, grid_y, grid_z), method='linear')
    #values = img_mask_data[points]
    #img_mask_data
    #del point_list
    grid_num = np.nan_to_num(grid_z2)
    show(grid_num[:,1,:])
    return grid_num

In [ ]:
def trying_4D(img):
    data = img.get_fdata()
    hdr = img.header
    hdr.get_xyzt_units()
    raw = hdr.structarr
    raw['xyzt_units']
    img = nib.Nifti1Image(data, np.eye(4))
    img.to_filename(os.path.join('build','test4d.nii.gz'))
    return

### Przystepujemy do mojej uproszczonej liniowej interpolacji



In [50]:
def simple_interpolation(img_mask_data):
    new_imgs = []
    for i in tqdm(range(5, img_mask_data.shape[1],5)):
        base_img1 = img_mask_data[:,i,:]
        base_img0 = img_mask_data[:,i-5,:]
        diff = (base_img1 - base_img0)
        diff_atomic = diff/5.0
        new_imgs.append(base_img0)
        for j in range(1, 5):
            interp = (base_img0 + (diff_atomic*j)).astype(int)
            new_imgs.append(interp)
    for k in range(i,img_mask_data.shape[1]):
        interp = (base_img1 + (diff_atomic*(img_mask_data.shape[1]-i))).astype(int)
        new_imgs.append(np.array(interp))
        
    return np.transpose(np.dstack(new_imgs),(0,2,1))


In [51]:
interp_mask = simple_interpolation(img_mask_data)

100%|██████████| 80/80 [00:00<00:00, 455.98it/s]
